We start with the data from 2023 and 2025, because we saw a big change in the political field between those two elections. For now as a simplification we will divide the parties into three groups:
- Left: GL_PvdA, SP, PvdD, Volt, DENK, BIJ1, CU
- Centre: NSC, VVD, D66, 50Plus
- Right: FvD, JA21, BVNL, PVV, BBB, SGP

#### Vote share
Luckily for us the vote share is known for each of the years and does not need to be calculated manually.

(The gap between BIJ 1, which is the party with the lowest amount of votes and the next party is a factor of 4. So BIJ1 is the cutoff point. The remaining parties not included here have a total of 0.4% of the total votes)

In [15]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [16]:
parties2023 = {
    'GL_PvdA': 0.1575, 'SP': 0.0315, 'PvdD': 0.0225,
    'Volt': 0.0171, 'DENK': 0.0237, 'BIJ1': 0.0042,
    'CU': 0.0204, 'NSC': 0.1288, 'VVD': 0.1524,
    'D66': 0.0629, '50Plus': 0.0049, 'FvD': 0.0223,
    'JA21': 0.0068, 'BVNL': 0.0051, 'PVV': 0.2349,
    'BBB': 0.0465, 'SPG': 0.0208
}

# Create left,right, centre list
left_keys = ['GL_PvdA', 'SP', 'PvdD', 'Volt', 'DENK', 'BIJ1', 'CU']
centre_keys = ['NSC', 'VVD', 'D66', '50Plus']
right_keys = ['FvD', 'JA21', 'BVNL', 'PVV', 'BBB', 'SPG']

In [17]:
def dict_to_party_arrays(party_share_dict):
    # Extract the parties from the list
    # Convert into a numpy array
    # Normalize probabilities so that all probabilities sum to 1
    parties = list(party_share_dict.keys())
    probs = np.array([party_share_dict[p] for p in parties], dtype=float)
    probs /= probs.sum()

    # Create mapping
    # Party to index and index to party
    p2i = {p: i for i, p in enumerate(parties)}
    i2p = {i: p for p, i in p2i.items()}
    return parties, probs, p2i, i2p

party_order = left_keys + centre_keys + right_keys
parties, probs_2023, p2i, i2p = dict_to_party_arrays(parties2023)

### ABM

Watts Strogatz documentation
- https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.watts_strogatz_graph.html
- https://www.geeksforgeeks.org/python/small-world-model-using-python-networkx/

Scale Free documentation
- https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.barabasi_albert_graph.html
- https://www.geeksforgeeks.org/dsa/barabasi-albert-graph-scale-free-models/

Random Graph documentation:
- https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.erdos_renyi_graph.html#networkx.generators.random_graphs.erdos_renyi_graph
- https://www.geeksforgeeks.org/dsa/erdos-renyl-model-generating-random-graphs/

In [18]:
"""
Create a function that creates different types of graphs.
For now the values assigned to the arguments of each graph are arbitrary.
"""
def build_graph(N, graph_type ='small_world', seed = 0, **kwargs):
    if graph_type =='small_world':
        k = kwargs.get('k', 10)
        p = kwargs.get('p', 0.05)
        return nx.watts_strogatz_graph(N,k,p, seed=seed)

    if graph_type == 'scale_free':
        m = kwargs.get('m', 5)
        return nx.barabasi_albert_graph(N,m , seed=seed)

    if graph_type =='random':
        p = kwargs.get('p', 0.002)
        return nx.erdos_renyi_graph(N,p,seed=seed)

In [19]:
"""
Use an adjacency list to reduce computation time.
We also need to make sure that the neighbor order remains the same.
"""
def adjacency_list(G):
    # total number of voters
    # create empty array
    N = G.number_of_nodes()
    adj = [None] * N

    # Convert neighbors of i into an array of integers
    for i in range(N):
        adj[i] = np.fromiter(G.neighbors(i), dtype=int)
    return adj



In [20]:
"""
We need to initialise the agents.
For now we will give them an arbitrary distribution
for loyalty and susceptibility, later on we can change this.
"""
def init_agents(N, probs, L_range=(0.7, 0.95), S_range=(0.3, 1.0), seed=0):
    rng = np.random.default_rng(seed)
    P = len(probs)

    # Assign parties to voters based on the 2023 vote share
    # Assign both loyalty and susceptibility
    party = rng.choice(P, size=N, p=probs)
    L = rng.uniform(L_range[0], L_range[1], size=N)
    S = rng.uniform(S_range[0], S_range[1], size=N)
    return party, L, S


In [21]:
"""
Create some way to store the atractiveness A of a certain party p at time t.
Baseline A = 1
Simulate events that alter the atractiveness of party p over a certain time
with a certain strength
i.e NSC: time_start = 5, time_end, A = 2.5
"""
def attractiveness_tracker(parties, T, events = None):
    # Set the initial attractiveness to 1
    # Float since the attracteveness can be a decimal number
    P = len(parties)
    A = np.ones((T, P), dtype=float)

    if events:
        # map party name to index
        name2idx = {p: i for i, p in enumerate(parties)}

        # Find the party which is effected by the events
        # Update the A for the time window
        for ev in events:
            idx = name2idx[ev["party"]]
            A[ev["start"]:ev["end"], idx] = float(ev["A"])

    return A


In [22]:
"""
This calcules the fi(t) from the model decription.
It calculates the fraction of neighbors of votes i which support party p.
"""
def neighbor_fraction_party(party, neighbors, num_parties):
    # find the amount of neighbors
    degree = len(neighbors)

    # special case if degree == 0
    # voter has no neighbors so environment has no influence on this votes
    if degree == 0:
        return np.ones(num_parties) / num_parties

    # get the party label of the neighbors
    # count how many neighbors belong to each party
    # return the normalized probability so that it sums to 1
    neigh_parties = party[neighbors]
    counts = np.bincount(neigh_parties, minlength=num_parties).astype(float)
    return counts/ counts.sum()



In [23]:
"""
Compute the probability based on the score (formula mentioned in project plan).
Uses the neighbor fraction calculated above.
Also uses attractiveness, susceptibility, Beta, Gamma.
"""
def compute_probs(
        neighbor_fraction, susceptibility,
        peer_pressure_strength, events_strength, attractiveness
    ):
    # compute scores and normalizee
    scores = np.exp(
        peer_pressure_strength * susceptibility * neighbor_fraction
        + events_strength * np.log(attractiveness)
    )
    scores /= scores.sum()

    return scores


In [24]:
"""
We must create a function that updates voter i.
First we must check the loyalty value.
Then we compute fi(t) from the neighbors.
Compute Pi(p) from f and A(t)
Sample a new party from Pi(p)
"""
def update_voter(
        voter_index, party, loyalty, susceptibility, neighbors,
        peer_pressure_strength, events_strength, attractiveness, rng
    ):
    # voter keeps the same party if rng is smaller than loyalty
    if rng.random < loyalty(voter_index):
        return

    # compute f
    num_parties = len(attractiveness)
    neighbor_votes = neighbor_fraction_party(
        party, neighbors[voter_index], num_parties
    )

    #Compute Pi(p)
    probs = compute_probs(
        neighbor_votes, susceptibility[voter_index],
        peer_pressure_strength, events_strength, attractiveness
    )

    #Sample new party
    party[voter_index] = rng.choice(num_parties, p=probs)


### Todo-list
- create simulation function
- recording/ storing function
- Plotting
- Grouping function (left,right,centre)
